# Libraries

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from nltk.stem import PorterStemmer
from textblob import TextBlob
from datasets import load_dataset
import pandas as pd
import numpy as np


#here we will download the dataset and some tools that we will need

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
paws_dataset = load_dataset('paws-x', 'en')

Dataset({
    features: ['id', 'sentence1', 'sentence2', 'label'],
    num_rows: 49401
})

In [ ]:
paws_dataset["train"][[1]]# to view the structure of the data

{'id': [2],
 'sentence1': ['The NBA season of 1975 -- 76 was the 30th season of the National Basketball Association .'],
 'sentence2': ['The 1975 -- 76 season of the National Basketball Association was the 30th season of the NBA .'],
 'label': [1]}

In [ ]:
train_data=paws_dataset['train']

In [ ]:
paws_dataframe=pd.DataFrame(train_data)# we wil process it as a dataframe

In [ ]:
paws_dataframe

,id,sentence1,sentence2,label
0,1,"In Paris , in October 1560 , he secretly met t...","In October 1560 , he secretly met with the Eng...",0
1,2,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,1
2,3,"There are also specific discussions , public p...","There are also public discussions , profile sp...",0
3,4,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,1
4,5,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,1
...,...,...,...,...
49396,49397,""" Our school is of spiritual and spiritual , l...",""" Our School is of the Temporal and the Spirit...",0
49397,49398,She was in Cork on June 24 and arrived on 8 Ju...,"She was at Cork on 24 June , and arrived in th...",1
49398,49399,Cornelia Stuyvesant Vanderbilt ( George and Ed...,John John F. A. Cecil ( the only child of Geor...,0
49399,49400,The third season was premiered on 7 June 2010 ...,"The fourth season was premiered on June 7 , 20...",0


#tokenization will be applied using word_tokenize

In [ ]:
tokenized_paws_dataframe=paws_dataframe.copy()
tokenized_paws_dataframe['sentence1'] = tokenized_paws_dataframe['sentence1'].apply(word_tokenize)
tokenized_paws_dataframe['sentence2'] = tokenized_paws_dataframe['sentence2'].apply(word_tokenize)
tokenized_paws_dataframe

,id,sentence1,sentence2,label
0,1,"[In, Paris, ,, in, October, 1560, ,, he, secre...","[In, October, 1560, ,, he, secretly, met, with...",0
1,2,"[The, NBA, season, of, 1975, --, 76, was, the,...","[The, 1975, --, 76, season, of, the, National,...",1
2,3,"[There, are, also, specific, discussions, ,, p...","[There, are, also, public, discussions, ,, pro...",0
3,4,"[When, comparable, rates, of, flow, can, be, m...","[The, results, are, high, when, comparable, fl...",1
4,5,"[It, is, the, seat, of, Zerendi, District, in,...","[It, is, the, seat, of, the, district, of, Zer...",1
...,...,...,...,...
49396,49397,"[``, Our, school, is, of, spiritual, and, spir...","[``, Our, School, is, of, the, Temporal, and, ...",0
49397,49398,"[She, was, in, Cork, on, June, 24, and, arrive...","[She, was, at, Cork, on, 24, June, ,, and, arr...",1
49398,49399,"[Cornelia, Stuyvesant, Vanderbilt, (, George, ...","[John, John, F., A., Cecil, (, the, only, chil...",0
49399,49400,"[The, third, season, was, premiered, on, 7, Ju...","[The, fourth, season, was, premiered, on, June...",0


In [ ]:
cleaned_tokenized_paws_dataframe=tokenized_paws_dataframe.copy()# we took an instance of the dataframe to apply tokeniaztion
cleaned_tokenized_paws_dataframe

,id,sentence1,sentence2,label
0,1,"[In, Paris, ,, in, October, 1560, ,, he, secre...","[In, October, 1560, ,, he, secretly, met, with...",0
1,2,"[The, NBA, season, of, 1975, --, 76, was, the,...","[The, 1975, --, 76, season, of, the, National,...",1
2,3,"[There, are, also, specific, discussions, ,, p...","[There, are, also, public, discussions, ,, pro...",0
3,4,"[When, comparable, rates, of, flow, can, be, m...","[The, results, are, high, when, comparable, fl...",1
4,5,"[It, is, the, seat, of, Zerendi, District, in,...","[It, is, the, seat, of, the, district, of, Zer...",1
...,...,...,...,...
49396,49397,"[``, Our, school, is, of, spiritual, and, spir...","[``, Our, School, is, of, the, Temporal, and, ...",0
49397,49398,"[She, was, in, Cork, on, June, 24, and, arrive...","[She, was, at, Cork, on, 24, June, ,, and, arr...",1
49398,49399,"[Cornelia, Stuyvesant, Vanderbilt, (, George, ...","[John, John, F., A., Cecil, (, the, only, chil...",0
49399,49400,"[The, third, season, was, premiered, on, 7, Ju...","[The, fourth, season, was, premiered, on, June...",0


#cleaning part (its done in 2 loops for some hardware limitations)

In [ ]:
def clean_text(tokens):


    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]  # Stopword Removal

    tokens = [token.lower() for token in tokens]  # Lowercasing

    # Removing special characters and other noise, keeping digits
    tokens = [token for token in tokens if token.isalnum()]

    return tokens


In [ ]:
errors = []  # List to store indices where errors occurred

for i in range(49401):
    try:

        sentence = tokenized_paws_dataframe['sentence1'][i]

        cleaned_sentence = clean_text(sentence)



        cleaned_tokenized_paws_dataframe['sentence1'][i] = cleaned_sentence
    except Exception as e:
        errors.append(i)  # Store the index where the error occurred
        print(f"Error at iteration {i}: {e}")

print("Iterations with errors:", errors)


In [ ]:
errors = []  # List to store where errors occurred

for i in range(49401):
    try:

        sentence = tokenized_paws_dataframe['sentence2'][i]
        print("the sentence before sending",sentence)
        cleaned_sentence = clean_text(sentence)



        cleaned_tokenized_paws_dataframe['sentence2'][i] = cleaned_sentence
    except Exception as e:
        errors.append(i)  # Store the index where the error occurred
        print(f"Error at iteration {i}: {e}")

print("Iterations with errors:", errors)


In [ ]:
tokenized_paws_dataframe # snippet of the text after cleaning

,id,sentence1,sentence2,label
0,1,"[In, Paris, ,, in, October, 1560, ,, he, secre...","[In, October, 1560, ,, he, secretly, met, with...",0
1,2,"[The, NBA, season, of, 1975, --, 76, was, the,...","[The, 1975, --, 76, season, of, the, National,...",1
2,3,"[There, are, also, specific, discussions, ,, p...","[There, are, also, public, discussions, ,, pro...",0
3,4,"[When, comparable, rates, of, flow, can, be, m...","[The, results, are, high, when, comparable, fl...",1
4,5,"[It, is, the, seat, of, Zerendi, District, in,...","[It, is, the, seat, of, the, district, of, Zer...",1
...,...,...,...,...
49396,49397,"[``, Our, school, is, of, spiritual, and, spir...","[``, Our, School, is, of, the, Temporal, and, ...",0
49397,49398,"[She, was, in, Cork, on, June, 24, and, arrive...","[She, was, at, Cork, on, 24, June, ,, and, arr...",1
49398,49399,"[Cornelia, Stuyvesant, Vanderbilt, (, George, ...","[John, John, F., A., Cecil, (, the, only, chil...",0
49399,49400,"[The, third, season, was, premiered, on, 7, Ju...","[The, fourth, season, was, premiered, on, June...",0


# the sentence1 and sentence2 stores the sentence as a list of words and we want it to be a full sentemce so we will concate the strings to form a sentence

In [ ]:
concatenated_cleaned_tokenized_paws_dataframe=cleaned_tokenized_paws_dataframe.copy()
concatenated_cleaned_tokenized_paws_dataframe

,id,sentence1,sentence2,label
0,1,"[paris, october, 1560, secretly, met, english,...","[october, 1560, secretly, met, english, ambass...",0
1,2,"[nba, season, 1975, 76, 30th, season, national...","[1975, 76, season, national, basketball, assoc...",1
2,3,"[also, specific, discussions, public, profile,...","[also, public, discussions, profile, specific,...",0
3,4,"[comparable, rates, flow, maintained, results,...","[results, high, comparable, flow, rates, maint...",1
4,5,"[seat, zerendi, district, akmola, region]","[seat, district, zerendi, akmola, region]",1
...,...,...,...,...
49396,49397,"[school, spiritual, spiritual, love, rehit, te...","[school, temporal, spiritual, love, rehit, spi...",0
49397,49398,"[cork, june, 24, arrived, 8, july, downs]","[cork, 24, june, arrived, downs, 8, july]",1
49398,49399,"[cornelia, stuyvesant, vanderbilt, george, edi...","[john, john, cecil, child, george, cornelia, s...",0
49399,49400,"[third, season, premiered, 7, june, 2010, four...","[fourth, season, premiered, june, 7, 2010, lik...",0


In [ ]:
concatenated_cleaned_tokenized_paws_dataframe['sentence1'] = concatenated_cleaned_tokenized_paws_dataframe['sentence1'].apply(lambda x: ' '.join(x))
concatenated_cleaned_tokenized_paws_dataframe['sentence2'] = concatenated_cleaned_tokenized_paws_dataframe['sentence2'].apply(lambda x: ' '.join(x))
concatenated_cleaned_tokenized_paws_dataframe


,id,sentence1,sentence2,label
0,1,paris october 1560 secretly met english ambass...,october 1560 secretly met english ambassador n...,0
1,2,nba season 1975 76 30th season national basket...,1975 76 season national basketball association...,1
2,3,also specific discussions public profile debat...,also public discussions profile specific discu...,0
3,4,comparable rates flow maintained results high,results high comparable flow rates maintained,1
4,5,seat zerendi district akmola region,seat district zerendi akmola region,1
...,...,...,...,...
49396,49397,school spiritual spiritual love rehit temporal...,school temporal spiritual love rehit spiritual...,0
49397,49398,cork june 24 arrived 8 july downs,cork 24 june arrived downs 8 july,1
49398,49399,cornelia stuyvesant vanderbilt george edith va...,john john cecil child george cornelia stuyvesa...,0
49399,49400,third season premiered 7 june 2010 fourth seas...,fourth season premiered june 7 2010 like third...,0


# now we will extract the sentences from the dataframe and store it all in one list called corpus

In [ ]:
sentence_1_text=c['sentence1'].tolist()
sentence_2_text=concatenated_cleaned_tokenized_paws_dataframe['sentence2'].tolist()

In [ ]:
sentence_1_text

['paris october 1560 secretly met english ambassador nicolas throckmorton asking passport return england scotland',
 'nba season 1975 76 30th season national basketball association',
 'also specific discussions public profile debates project discussions',
 'comparable rates flow maintained results high',
 'seat zerendi district akmola region',
 'william henry henry harman born 17 february 1828 waynesboro virginia parents lewis sally garber harman',
 'bullion express concept introduced new store dallas texas preston center opened',
 'discrete amount probabilities formula 1 condition formula 2 formula 3 real number tsallis defined entropy',
 'soviet union maintained embassy oslo consulate barentsburg norway maintained message moscow',
 'vocabulary even went brazil leaving portuguese settlers macanese chinese settlers',
 'kabir suman recorded several albums name suman chattopaddhyay suman chatterjee 1992 1999',
 'scholar metaphysical literature theology classical sciences',
 'city sits co

In [ ]:
sentence_2_text

['october 1560 secretly met english ambassador nicolas throckmorton paris asked passport return scotland england',
 '1975 76 season national basketball association 30th season nba',
 'also public discussions profile specific discussions project discussions',
 'results high comparable flow rates maintained',
 'seat district zerendi akmola region',
 'william henry harman born waynesboro virginia february 17 1828 parents lewis sally garber harman',
 'bullion express concept introduced new store opened preston center dallas texas',
 'given discrete set probabilities formula 1 condition formula 2 formula 3 real number tsallis entropy defined',
 'soviet union maintained embassy moscow consulate barentsburg norway maintained message oslo',
 'vocabulary even went brazil leaving macanese chinese settlers portuguese settlers',
 'suman chatterjee recorded number albums 1992 1999 name suman chattopaddhyay kabir suman',
 'scholar metaphysical literature theology classical science',
 'city lies conf

In [ ]:
corpus=sentence_1_text+sentence_2_text
corpus

['paris october 1560 secretly met english ambassador nicolas throckmorton asking passport return england scotland',
 'nba season 1975 76 30th season national basketball association',
 'also specific discussions public profile debates project discussions',
 'comparable rates flow maintained results high',
 'seat zerendi district akmola region',
 'william henry henry harman born 17 february 1828 waynesboro virginia parents lewis sally garber harman',
 'bullion express concept introduced new store dallas texas preston center opened',
 'discrete amount probabilities formula 1 condition formula 2 formula 3 real number tsallis defined entropy',
 'soviet union maintained embassy oslo consulate barentsburg norway maintained message moscow',
 'vocabulary even went brazil leaving portuguese settlers macanese chinese settlers',
 'kabir suman recorded several albums name suman chattopaddhyay suman chatterjee 1992 1999',
 'scholar metaphysical literature theology classical sciences',
 'city sits co

# finally qwe will save the cleaned text in a csv file for the next part

In [ ]:
df.to_csv('data.csv', index=False).to_csv('data.csv', index=False)